In [5]:
import requests
import json
from bs4 import BeautifulSoup as bts
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import os
import datetime

In [6]:
# 네이버뉴스 링크를 수집하고 데이터프레임으로 반환하는 함수 생성
def NaverNewsLink(searchWord, startNo = 1, startDate='', endDate=''):
    
    # HTTP 요청 실행
    # 쿼리 문자열: 정렬(sort)-관련도순, 기간(pd)-전체
    res = requests.get(
        url = 'https://s.search.naver.com/p/newssearch/search.naver',         
        params = {
            'nqx_theme': {'theme':{'main':{'name':'sports_event'},'sub':[{'name':'issue'}]}},
            'query': searchWord,
            'sort': 0,
            'spq': 3,
            'pd': 0,
            'start': startNo,
            'where': 'news_tab_api',
            'ds': startDate,
            'de': endDate 
        }
    )
    
    # JSON 형태의 문자열을 딕셔너리로 변환
    dic = json.loads(s = res.text)
    
    # 딕셔너리의 각 원소를 bs4.BeautifulSoup 객체로 변환
    # [주의] HTTP 응답 바디 문자열에 공백이 있으면 items의 원소로 추가되므로 삭제해야 함
    items = [bts(markup = i.strip(), features = 'html.parser') for i in dic['contents']]
    
    # 뉴스 링크를 포함하는 HTML 요소를 원소로 갖는 links 생성
    links = [item.select('a.info:last-child') for item in items]

    # 기사 작성 일시를 포함하는 HTML 요소를 원소로 갖는 dates 생성
    dates = [item.select('span.info')[0].text for item in items if len(item.select('span.info')) > 0]
    
    # 언론사, 제목 및 네이버뉴스 링크를 원소로 갖는 데이터프레임 생성
    newsList = pd.DataFrame(data = {
        'press': [item.select(selector = 'a.press')[0].text for item in items], 
        'title': [item.select(selector = 'a.news_tit')[0].text for item in items], 
        'nlink': [link[0]['href'] if len(link) >= 1 else np.nan for link in links],
        'date': dates
    })
    
    # 데이터프레임 반환
    return newsList

In [3]:
# 특정 검색어와 날짜 범위를 설정하고 1년 동안의 기사를 수집하는 함수
def collect_year_news(searchWord):
    newsList = pd.DataFrame() # 결과 저장할 빈 프레임
    end_date = datetime.datetime.now() # 오늘
    start_date = end_date - datetime.timedelta(days=365) # 1년전 날짜
    
    while start_date <= end_date: # 1년치 뉴스 수집
        # 날짜별 뉴스 수집
        df = NaverNewsLink(
            searchWord, 
            startNo=1,  # 첫 번째 페이지에서 시작 (기본 10개 기사 가져오기)
            startDate=start_date.strftime('%Y.%m.%d'), 
            endDate=start_date.strftime('%Y.%m.%d')
        )
        
        # 수집한 데이터를 newsList에 추가
        newsList = pd.concat([newsList, df], ignore_index=True)
        
        # 1초간 멈춤 (너무 빠르게 요청하지 않도록)
        time.sleep(1)
        
        # 하루씩 증가
        start_date += datetime.timedelta(days=1)
    
    return newsList

In [1]:
# 검색어 설정
searchWord = '삼성전자'

In [7]:
newsList = collect_year_news(searchWord)
#print(newsList.tail(10))

In [8]:
newsList.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3660 entries, 0 to 3659
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   press   3660 non-null   object
 1   title   3660 non-null   object
 2   nlink   3657 non-null   object
 3   date    3660 non-null   object
dtypes: object(4)
memory usage: 114.5+ KB


In [9]:
# press에서 '언론사 선정' 삭제
newsList['press'] = newsList['press'].str.replace(pat = '언론사 선정', repl = '')

In [10]:
# press 처음 5행 확인
newsList['press'].head()

0    TV조선
1    경향신문
2    연합뉴스
3     뉴스1
4     뉴시스
Name: press, dtype: object

In [11]:
# nlink가 결측인 행 삭제
newsList = newsList.loc[newsList['nlink'].notna(), :]

In [12]:
# newsList의 행 개수 확인
newsList.shape[0]

3657

In [13]:
# nlink의 중복 여부(True/False)를 원소로 갖는 dups 생성
# [참고] keep 매개변수에 'first'를 지정하면 첫 번째 중복 건은 False로 반환
dups = newsList['nlink'].duplicated(keep = 'first')

In [14]:
# nlink가 중복(dups가 True)인 건수 확인
dups.sum() 

np.int64(3636)

In [15]:
# nlink가 중복인 행이 있으면 해당 행을 삭제
#newsList = newsList.loc[~dups, :]

In [16]:
# newsList의 행 개수 확인
newsList.shape[0]

3657

In [17]:
# nlink에서 쿼리 문자열 삭제
newsList['nlink'] = newsList['nlink'].str.replace(pat = r'(\?.+)', repl = '', regex = True)

In [18]:
# newsList의 처음 5행 확인
newsList.tail()

,press,title,nlink,date
3655,한국경제TV,"삼성전자, 저전력 특화 '24Gb GDDR7 D램' 개발",https://n.news.naver.com/mnews/article/215/000...,2시간 전
3656,지디넷코리아,"삼성전자, 28일 갤럭시북5 프로 360 국내 출시",https://n.news.naver.com/mnews/article/092/000...,1일 전
3657,노컷뉴스,"삼성전자, AI PC '갤럭시 북5 프로 360' 28일 국내 출시",https://n.news.naver.com/mnews/article/079/000...,1일 전
3658,뉴시스,"삼성전자·SK하이닉스 강세…""TSMC 실적 촉각""[핫스탁]",https://n.news.naver.com/mnews/article/003/001...,2시간 전
3659,세계일보,삼성전자에 발등 찍힌 국민연금 [경제 레이더],https://n.news.naver.com/mnews/article/022/000...,A14면 1단


In [19]:
# newsList의 행이름 초기화
newsList = newsList.reset_index(drop = True)

In [27]:
# newsList의 처음 5행 확인
newsList.head()

,press,title,nlink,date
0,TV조선,"삼성전자, 업계 최초 '24Gb GDDR7 D램' 개발",https://n.news.naver.com/mnews/article/448/000...,2시간 전
1,경향신문,"삼성전자, 역대급으로 얇은 ‘갤럭시 Z폴드SE’ 21일 공개",https://n.news.naver.com/mnews/article/032/000...,30분 전
2,연합뉴스,"삼성전자, 업계 최초 24Gb GDDR7 D램 개발…내년 초 제품 상용화",https://n.news.naver.com/mnews/article/001/001...,2시간 전
3,뉴스1,"""문틈도 가뿐히 통과""…삼성전자, 갤폴드 슬림 21일 공개",https://n.news.naver.com/mnews/article/421/000...,32분 전
4,뉴시스,"삼성전자·SK하이닉스 강세…""TSMC 실적 촉각""[핫스탁]",https://n.news.naver.com/mnews/article/003/001...,1시간 전


In [21]:
# 현재 작업 경로 확인
os.getcwd()

'C:\\Users\\2-13\\Desktop\\DMF\\final\\naver'

In [22]:
# newsList를 pkl 파일로 저장
pd.to_pickle(obj = newsList, filepath_or_buffer = 'Naver_News_List.pkl')

In [23]:
# 현재 작업 경로에 있는 폴더명과 파일명 확인
os.listdir()

['.ipynb_checkpoints',
 'crawling.py',
 'data',
 'Naver_News_List.pkl',
 'naver_news_samsung.csv',
 '삼전 크롤링 step2.ipynb',
 '삼전크롤링.ipynb']

## end of document